In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import requests

twitter_dataset_url = "https://raw.githubusercontent.com/mitjat/langid_eval/master/uniformly_sampled.tsv"
twitter_dataset = 'uniformly_sampled.tsv'

In [6]:
import os 
if twitter_dataset not in os.listdir('.'): 
    r = requests.get(twitter_dataset_url)
    tweets = None
    if r.status_code == 200: 
        tweets = r.text

In [8]:
# data is just text: needs massaging
tweets[:100]

u'ar\t483762194908479488\nar\t483762916097654784\nar\t483764454283558912\nar\t483764828784582656\nar\t483765526'

In [11]:
lines = tweets.split('\n')
langs = []
ids = []
tweet_data = []
for l in lines: 
    l = l.strip()
    if l is None: 
        continue
    parts = l.split('\t')
    if len(parts) == 2:
        langs.append(parts[0])
        ids.append(parts[1])
tweet_data = zip(langs, ids)

In [12]:
tweet_data[:10]

[(u'ar', u'483762194908479488'),
 (u'ar', u'483762916097654784'),
 (u'ar', u'483764454283558912'),
 (u'ar', u'483764828784582656'),
 (u'ar', u'483765526683209728'),
 (u'ar', u'483768342315282432'),
 (u'ar', u'483770689933946880'),
 (u'ar', u'483770765985083392'),
 (u'ar', u'483770900127285248'),
 (u'ar', u'483770997892345857')]

In [14]:
df = pd.DataFrame(data=tweet_data, columns=['language', 'tweet_id'])
df.to_csv(twitter_dataset, index=False, header=False)

In [15]:
# inspect data
df

,language,tweet_id
0,ar,483762194908479488
1,ar,483762916097654784
2,ar,483764454283558912
3,ar,483764828784582656
4,ar,483765526683209728
5,ar,483768342315282432
6,ar,483770689933946880
7,ar,483770765985083392
8,ar,483770900127285248
9,ar,483770997892345857


In [16]:
df.describe()

,language,tweet_id
count,120571,120571
unique,84,120511
top,en,0
freq,37817,61


In [22]:
# this means that: 
# num unique language classes: 84 -- including "und"
# num ids : 120571 -- and they both match up, so we massaged the data properly
# most freq lang: en -- to be expected
# most freq tweet_id: 0 -- hmm...

In [21]:
# it's strange that the most frequent tweet_id is 0
# so we should check that 
df.loc[df['tweet_id'] == '0']

,language,tweet_id
5324,en,0
43141,es,0
43142,es,0
43143,es,0
43144,es,0
43145,es,0
43146,es,0
43147,es,0
43148,es,0
43149,es,0


In [28]:
# we should probably remove those because they seem like bad data
# and we want to download the tweets properly
# see: http://pandas.pydata.org/pandas-docs/stable/10min.html#min
df[df.tweet_id != '0'].count()

language    120510
tweet_id    120510
dtype: int64

In [29]:
# this looks about right, so we put it into a new clean frame
df_cleaned = df[df.tweet_id != '0']

In [31]:
cleaned_file = 'twitter_cleaned_uniformly_sampled.tsv'
df_cleaned.to_csv(cleaned_file, sep='\t')

In [ ]:
#
# Now we deal with downloading the tweets 
# Since this involves OAuth, we'll do this offline 
# And upload the data later
#